# Abnormal Enviornment Detector

The "**Calibration**" portion records data and averages it to determine a "normal" benchmarks of each reading of the environmental sensor. And the "**Check Changes**" portion actively in live time checks the environment for any deviations of those norms.

---

## Calibration

### Setup

In [1]:
import bme680
import time
import numpy as np
from matplotlib import pyplot as plt

sensor = bme680.BME680()

sensor.set_humidity_oversample(bme680.OS_2X)
sensor.set_pressure_oversample(bme680.OS_4X)
sensor.set_temperature_oversample(bme680.OS_8X)
sensor.set_filter(bme680.FILTER_SIZE_3)

sensor.set_gas_status(bme680.ENABLE_GAS_MEAS)
sensor.set_gas_heater_temperature(320)
sensor.set_gas_heater_duration(150)
sensor.select_gas_heater_profile(0)

### Calibrating

In [2]:
# Finding 'normal' values

# number of minutes to calibrate
cal_time = 5

temp = []
pres = []
hum = []
res = []
sensor.get_sensor_data() # preliminary runs to ensure the other checks work

for i in range(cal_time):
    
    if sensor.get_sensor_data():
        temp.append(sensor.data.temperature)
        pres.append(sensor.data.pressure)
        hum.append(sensor.data.humidity)
        
        if sensor.data.heat_stable:
            res.append(sensor.data.gas_resistance)
            
        else:
            print("Unable to collect gas resistance")
            
    else:
        print("Could not collect data. Skipping this check.")
        continue
    
    time.sleep(60)

avg_temp = np.average(temp)
avg_pres = np.average(pres)
avg_hum = np.average(hum)
avg_res = np.average(res)

print(f"average temp: {avg_temp}")
print(f"average pressure: {avg_pres}")
print(f"average humidity: {avg_hum}")
print(f"average gas res.: {avg_res}")

Unable to collect gas resistance
average temp: 26.224
average pressure: 992.774
average humidity: 66.16080000000001
average gas res.: 8478.799611157425


## Check Changes

The code below runs in live time to check every `interval` seconds if the readings of the sensor deviate from the normal values calculated above. It will print a warning if they do. If not, it will state the values are normal. Exit the code by pressing the stop button at the top of the page. Side note: to get the readings to change, I held the sensor in my hands and slowly blew inside my hands.

In [3]:
interval = 5 # in seconds

try:
    while True:
        if sensor.get_sensor_data():

            ########## Temperature ##########
            t = sensor.data.temperature
            t_out = t < avg_temp - 2 or t > avg_temp + 2 # change the restriction values if needed

            if t_out:
                print("WARNING: Temperature is abnormal at " + str(t))
            else:
                print("Temperature is normal")

            ########## Air Presure ##########
            p = sensor.data.pressure
            p_out = p < avg_pres - 5 or p > avg_pres + 5 # change the restriction values if needed

            if p_out:
                print("WARNING: Pressure is abnormal at " + str(p))

            else:
                print("Pressure is normal")

            ########## Humidity ##########
            h = sensor.data.humidity
            h_out = h < avg_hum - 5 or h > avg_hum + 5 # change the restriction values if needed

            if h_out:
                print("WARNING: Humidity is abnormal at " + str(h))

            else:
                print("Humidity is normal")

            ########## Gas Resistance ##########
            if sensor.data.heat_stable:
                r = sensor.data.gas_resistance
                r_out = r < avg_res - 5 or h > avg_res + 5 # change the restriction values if needed

                if r_out:
                    print("WARNING: Gas Resistance is abnormal at " + str(r))

                else:
                    print("Gas Resistance is normal")

        time.sleep(interval)
except KeyboardInterrupt:
    print('Program stopped')

Temperature is normal
Pressure is normal
Humidity is normal
Temperature is normal
Pressure is normal
Humidity is normal
Temperature is normal
Pressure is normal
Humidity is normal
Gas Resistance is normal
Temperature is normal
Pressure is normal
Humidity is normal
Gas Resistance is normal
Temperature is normal
Pressure is normal
Gas Resistance is normal
Temperature is normal
Pressure is normal
Gas Resistance is normal
Pressure is normal
Gas Resistance is normal
Pressure is normal
Gas Resistance is normal
Pressure is normal
Gas Resistance is normal
Pressure is normal
Gas Resistance is normal
Pressure is normal
Gas Resistance is normal
Pressure is normal
Gas Resistance is normal
Pressure is normal
Gas Resistance is normal
Pressure is normal
Humidity is normal
Gas Resistance is normal
Pressure is normal
Humidity is normal
Gas Resistance is normal
Pressure is normal
Humidity is normal
Gas Resistance is normal
Pressure is normal
Humidity is normal
Gas Resistance is normal
Pressure is normal